# **Procesamiento de Lenguaje Natural**

## **Tarea 2 - Notebook 1**

## Integrantes

* ### Daniel Osorio Cárdenas
* ### Juan Diego Calixto Núñez

Este Notebook incluye los literales I, II, III, IV de la tarea. Estos corresponden a la construcción de los modelos de lenguaje que se usarán en el Notebook 2.

## **I. Creación de los archivos consolidados**

In [1]:
# Configurar los directorios donde se encuentran los archivos de datos
NEWS_FOLDER = '20news-18828'
BLOGS_FOLDER = 'blogs'

Empezamos cargando los archivos de 20news-18828 y creamos un único archivo consolidado llamado "20N_consolidated.txt".

In [2]:
import os

# Listar los folders de 20news-18828
news_folder = os.listdir(NEWS_FOLDER)

# Recorremos cada archivo de news
news_texts = []
for folder in news_folder:
    for file_name in os.listdir(NEWS_FOLDER + '/' + folder):
        extracted_text = ''
        with open(NEWS_FOLDER + '/' + folder + '/' + file_name, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                # Agregamos solo las lineas del contenido del archivo
                if not line.startswith('From:') and not line.startswith('Subject:') and not line.startswith('Fax:') and not line.startswith('Phone:') and not line.startswith('Email:') and not line.startswith('INTERNET:'):
                    extracted_text += line + " "
            news_texts.append(extracted_text)

# Guardamos todos los textos en un unico .txt
with open('20N_consolidated.txt', 'w', encoding='utf-8') as file:
    for text in news_texts:
        file.write(text)

# Revisamos el tamaño del archivo .txt
print('Tamaño del archivo 20N_consolidated.txt: ', os.path.getsize('20N_consolidated.txt'), 'bytes = ', round(os.path.getsize('20N_consolidated.txt') / 1048576, 2), 'MB')

Tamaño del archivo 20N_consolidated.txt:  31568937 bytes =  30.11 MB


Para cargar los archivos de blogs se hizo primero un preprocesamiento de los archivos XML puesto que hubo muchos errores a la hora de leerlos. La estrategia fue la siguiente:
* Se eliminaron los tags de formato del XML
* Se eliminaron las fechas de los posts
* Se dejó unicamente el contenido de los posts

Esto para poder leer los archivos como texto plano. El archivo que hace esta modificación se llama "blogs_preprocessing.py".

Finalmente, se cargaron los archivos de blogs y se creó un único archivo consolidado llamado "BAC_consolidated.txt".

In [3]:
# Recorremos cada archivo de blogs
blogs_texts = []
for filename in os.listdir(BLOGS_FOLDER):
    extracted_text = ''
    with open(BLOGS_FOLDER + '/' + filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            # Agregamos solo las lineas del contenido del archivo
            if not line.startswith('From:') and not line.startswith('Subject:') and not line.startswith('Fax:') and not line.startswith('Phone:') and not line.startswith('Email:') and not line.startswith('INTERNET:'):
                extracted_text += line + " "
        blogs_texts.append(extracted_text)

# Guardamos todos los textos en un unico .txt
with open('BAC_consolidated.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts:
        file.write(text)

# Revisamos el tamaño del archivo .txt
print('Tamaño del archivo BAC_consolidated.txt: ', os.path.getsize('BAC_consolidated.txt'), 'bytes = ', round(os.path.getsize('BAC_consolidated.txt') / 1048576, 2), 'MB')

Tamaño del archivo BAC_consolidated.txt:  757389894 bytes =  722.3 MB


## **II y III. Crear archivos de Training y Test y Tokenizar por sentencia**

Primero se crean los archivos de training y test. Para esto se separó cada archivo consolidado por oraciones. Suponemos que las oraciones están separadas por puntos. Se crearon dos archivos por cada corpus, uno de training y otro de test. Los archivos de training contienen el 80% de las oraciones y los de test el 20% restante.

In [4]:
# Cargamos los textos de los archivos .txt
with open('20N_consolidated.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts = file.read()

with open('BAC_consolidated.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts = file.read()

# Separar cada archivo en oraciones
news_texts = news_texts.split('.')
blogs_texts = blogs_texts.split('.')

# Quitar los textos vacios
news_texts = [text for text in news_texts if text != '']
blogs_texts = [text for text in blogs_texts if text != '']

print('Cantidad de oraciones en 20news-18828: ', len(news_texts))
print('Cantidad de oraciones en BAC: ', len(blogs_texts))

# Dividir los textos en 80% train y 20% test
news_texts_train = news_texts[:int(len(news_texts) * 0.8)]
news_texts_test = news_texts[int(len(news_texts) * 0.8):]
blogs_texts_train = blogs_texts[:int(len(blogs_texts) * 0.8)]
blogs_texts_test = blogs_texts[int(len(blogs_texts) * 0.8):]

# Guardar los textos de train y test en archivos .txt
with open('20N_GROUP_training.txt', 'w', encoding='utf-8') as file:
    for text in news_texts_train:
        file.write(text.strip() + '. ')

with open('20N_GROUP_testing.txt', 'w', encoding='utf-8') as file:
    for text in news_texts_test:
        file.write(text.strip() + '. ')

with open('BAC_GROUP_training.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts_train:
        file.write(text.strip() + '. ')

with open('BAC_GROUP_testing.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts_test:
        file.write(text.strip() + '. ')

# Revisamos el tamaño de los archivos .txt
print('Tamaño del archivo .txt de 20N_GROUP_training: ', os.path.getsize('20N_GROUP_training.txt'), 'bytes = ', round(os.path.getsize('20N_GROUP_training.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de 20N_GROUP_testing: ', os.path.getsize('20N_GROUP_testing.txt'), 'bytes = ', round(os.path.getsize('20N_GROUP_testing.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de BAC_GROUP_training: ', os.path.getsize('BAC_GROUP_training.txt'), 'bytes = ', round(os.path.getsize('BAC_GROUP_training.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de BAC_GROUP_testing: ', os.path.getsize('BAC_GROUP_testing.txt'), 'bytes = ', round(os.path.getsize('BAC_GROUP_testing.txt') / 1048576, 2), 'MB')


Cantidad de oraciones en 20news-18828:  425274
Cantidad de oraciones en BAC:  9425954
Tamaño del archivo .txt de 20N_GROUP_training:  25266186 bytes =  24.1 MB
Tamaño del archivo .txt de 20N_GROUP_testing:  6265196 bytes =  5.97 MB
Tamaño del archivo .txt de BAC_GROUP_training:  599507899 bytes =  571.74 MB
Tamaño del archivo .txt de BAC_GROUP_testing:  148491073 bytes =  141.61 MB


Ahora procedemos a iniciar la tokenización. Se van a modelar los inicios de sentencia como \<s> y los finales como \</s>. Los numeros se van a reemplazar por \<NUM>. Finalmente, se van a reemplazar los términos con frecuencia 1 por \<UNK>. Para este último paso se creó un diccionario que contiene la frecuencia de cada término en el corpus de training mediante el uso de la función Counter de la librería collections.

A continuación se muestra un ejemplo con sentencias de prueba para mostrar cómo se tokenizan los textos.

In [2]:
from collections import Counter

In [3]:
# Se va a hacer una prueba de cómo se tokeniza una oración

# Primero generamos varias oraciones con palabras repetidas y con caracteres especiales, y con algunas palabras únicas
test_sentences = [  'Hola! soy Daniel Osorio y me gusta jugar al fubol',
                    'Hola! soy Daniel Osorio y me gusta jugar al fubol',
                    "Tambien me parece gusta jugar?",
                    "Tengo 20 años y me gusta jugar al fubol",
                    "¿Cuantos años tienes tu?",
                    "Tengo entre 20-40"]

# Se obtienen las frecuencias de cada palabra
test_word_counts = Counter()
for sentence in test_sentences:
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Si el ultimo caracter de una palabra no es una letra, se quita
    for i in range(len(sentence)):
        if sentence[i] != "" and not sentence[i][-1].isalpha():
            sentence[i] = sentence[i][:-1]
    # Se agregan las palabras al diccionario de frecuencias
    test_word_counts.update(sentence)

print(test_word_counts.most_common()[:-10-1:-1])

# Ahora se va a tokenizar cada oración
test_tokens = []
for sentence in test_sentences:
    # Primero se agrega el token de inicio de oracion
    test_tokens.append('<s>')
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Se van revisando las palabras de la sentencia
    for word in sentence:
        # Si la palabra es un numero, se agrega el token <NUM>
        if word.isnumeric():
            test_tokens.append('<NUM>')
        # Si la palabra no es vacia
        elif word != "":
            # Si la palabra termina en algo que no sea una letra, 
            # se quita el ultimo caracter de la palabra y se agregan las dos partes
            if not (word[-1].isalpha() or word[-1].isnumeric()):
                # Si la palabra restante es un numero, se agrega el token <NUM>
                if word[:-1].isnumeric():
                    test_tokens.append('<NUM>')
                # Si la palabra restante tiene frecuencia 1 o menos, se agrega el token <UNK>
                elif test_word_counts[word[:-1]] <= 1: # Se hace así por si la palabra restante tiene frecuencia 0 (no existe)
                    test_tokens.append('<UNK>')
                else:
                    test_tokens.append(word[:-1])
                test_tokens.append(word[-1])
            # Si su frecuencia es 1 o menos, se agrega el token <UNK>
            elif test_word_counts[word] <= 1:
                test_tokens.append('<UNK>')
            # Si su frecuencia es mayor a 1, se agrega la palabra
            else:
                test_tokens.append(word)
    # Se agrega el token de fin de oracion
    test_tokens.append('</s>')

print(test_tokens)

[('20-4', 1), ('entre', 1), ('tu', 1), ('tienes', 1), ('¿Cuantos', 1), ('2', 1), ('parece', 1), ('Tambien', 1), ('años', 2), ('Tengo', 2)]
['<s>', 'Hola', '!', 'soy', 'Daniel', 'Osorio', 'y', 'me', 'gusta', 'jugar', 'al', 'fubol', '</s>', '<s>', 'Hola', '!', 'soy', 'Daniel', 'Osorio', 'y', 'me', 'gusta', 'jugar', 'al', 'fubol', '</s>', '<s>', '<UNK>', 'me', '<UNK>', 'gusta', 'jugar', '?', '</s>', '<s>', 'Tengo', '<NUM>', 'años', 'y', 'me', 'gusta', 'jugar', 'al', 'fubol', '</s>', '<s>', '<UNK>', 'años', '<UNK>', '<UNK>', '?', '</s>', '<s>', 'Tengo', '<UNK>', '<UNK>', '</s>']


Se tokeniza el archivo 20N_GROUP_training.txt. Primero se encuentra la frecuencia de cada palabra para después reemplazar las palabras con frecuencia 1 por \<UNK>.

In [4]:
# Se cargan los archivos de training
with open('20N_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts_train = file.read()

# Vamos a separar el corpus de train de 20N en sentencias
news_texts_train = news_texts_train.split('. ')
print('Cantidad de oraciones en 20N_GROUP_training: ', len(news_texts_train))

# Vamos a recorrer cada sentencia y separarla en palabras para agregarlas a un diccionario de frecuencias
news_word_counts_train = Counter()
for sentence in news_texts_train:
    sentence = sentence.lower()
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Si el ultimo caracter de una palabra no es una letra, se quita
    for i in range(len(sentence)):
        if sentence[i] != "" and not sentence[i][-1].isalpha():
            sentence[i] = sentence[i][:-1]
    # Se agregan las palabras al diccionario de frecuencias
    news_word_counts_train.update(sentence)

# Se revisan las palabras menos frecuentes
print(news_word_counts_train.most_common()[:-10-1:-1])

Cantidad de oraciones en 20N_GROUP_training:  340220
[('haywood', 1), ('sheffield-hallam', 1), ('uptodate', 1), ('termcap', 1), ('terminfo', 1), ('tilde', 1), ('per-user', 1), ('xsetup_', 1), ('(~user', 1), ('~user', 1)]


Ahora se procede a tokenizar el texto, estos se guardan en una lista.

In [5]:
print('Cantidad de palabras en 20N_GROUP_training: ', len(news_word_counts_train))

# Ahora se va a terminar de tokenizar el corpus de train de 20N
news_tokens_train = []

for sentence in news_texts_train:
    # Primero se agrega el token de inicio de oracion
    news_tokens_train.append('<s>')
    # Se normaliza la sentencia
    sentence = sentence.lower()
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Se van revisando las palabras de la sentencia
    for word in sentence:
        # Si la palabra es un numero, se agrega el token <NUM>
        if word.isnumeric():
            news_tokens_train.append('<NUM>')
        # Si la palabra no es vacia
        elif word != "":
            # Si la palabra termina en algo que no sea una letra o un numero
            # se quita el ultimo caracter de la palabra y se agregan las dos partes
            if not (word[-1].isalpha() or word[-1].isnumeric()):
                # Si la palabra restante es un numero, se agrega el token <NUM>
                if word[:-1].isnumeric():
                    news_tokens_train.append('<NUM>')
                # Si la palabra restante tiene frecuencia 1 o menos, se agrega el token <UNK>
                elif news_word_counts_train[word[:-1]] <= 1: # Se hace así por si la palabra restante tiene frecuencia 0 (no existe)
                    news_tokens_train.append('<UNK>')
                else:
                    news_tokens_train.append(word[:-1])
                news_tokens_train.append(word[-1])
            # Si su frecuencia es 1 o menos, se agrega el token <UNK>
            elif news_word_counts_train[word] <= 1:
                news_tokens_train.append('<UNK>')
            # Si su frecuencia es mayor a 1, se agrega la palabra
            else:
                news_tokens_train.append(word)
    # Se agrega el token de fin de oracion
    news_tokens_train.append('</s>')

print('Cantidad de tokens en 20N_GROUP_training: ', len(news_tokens_train))

Cantidad de palabras en 20N_GROUP_training:  209342
Cantidad de tokens en 20N_GROUP_training:  5261998


In [5]:
print(news_tokens_train[:1000])

['<s>', 'the', 'm', 'code', 'stream', 'might', 'be', 'independently', 'attacked', 'based', 'on', 'knowledge', 'of', 'clipper', 'chip', 'protocols', 'as', 'revealed', 'plaintext', '</s>', '<s>', 'this', 'could', 'be', 'invalidated', 'by', 'changing', 'the', 'temporal', 'and', 'or', 'spatial', 'relationship', 'of', 'the', 'clipper', 'm', 'stream', 'and', 'the', 'actual', 'transmitted', 'stream', ',', 'under', 'the', 'control', 'of', 'a', 'secure', 'key', 'generator', 'synchronized', 'between', 'endpoints', '</s>', '<s>', 'the', 'useful', 'life', 'time', 'of', 'captured', 'law', 'enforcement', 'blocks', 'might', 'be', 'limited', 'based', 'on', 'hostile', 'forces', 'using', 'them', 'as', 'targets', 'following', 'transmission', 'interception', '</s>', '<s>', 'you', 'would', 'need', 'a', 'large', 'number', 'of', 'them', ',', 'but', ',', 'hey', "there's", 'supposed', 'to', 'be', 'millions', 'of', 'these', 'things', ',', 'right', '?', 'adding', 'time', 'stamps', 'to', 'the', 'encrypted', 'law'

Ahora se empieza obteniendo las frecuencias de cada palabra en el corpus de training. Se crea un diccionario que contiene la frecuencia de cada palabra.

In [2]:
with open('BAC_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts_train = file.read()

# Vamos a separar el corpus de train de BAC en sentencias
blogs_texts_train = blogs_texts_train.split('. ')
print('Cantidad de oraciones en BAC_GROUP_training: ', len(blogs_texts_train))

# Vamos a recorrer cada sentencia y separarla en palabras para agregarlas a un diccionario de frecuencias
blogs_word_counts_train = Counter()
for sentence in blogs_texts_train:
    sentence = sentence.lower()
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Si el ultimo caracter de una palabra no es una letra, se quita
    for i in range(len(sentence)):
        if sentence[i] != "" and not sentence[i][-1].isalpha():
            sentence[i] = sentence[i][:-1]
    # Se agregan las palabras al diccionario de frecuencias
    blogs_word_counts_train.update(sentence)

# Se revisan las palabras menos frecuentes
print(blogs_word_counts_train.most_common()[:-10-1:-1])

Cantidad de oraciones en BAC_GROUP_training:  7540764
[('working/learning/reading', 1), ('shimrat', 1), ('irit', 1), ('theme!&nbsp', 1), ('maelene', 1), ('excitted', 1), ('chimms', 1), ('taurens', 1), ('arugh!!', 1), ('sb510', 1)]


Ahora se tokeniza el texto. Se reemplazan los números por \<NUM> y las palabras con frecuencia 1 por \<UNK>.

In [3]:
print('Cantidad de palabras en BAC_GROUP_training: ', len(blogs_word_counts_train))

# Ahora se va a terminar de tokenizar el corpus de train de BAC
blogs_tokens_train = []

for sentence in blogs_texts_train:
    # Primero se agrega el token de inicio de oracion
    blogs_tokens_train.append('<s>')
    # Se normaliza la sentencia
    sentence = sentence.lower()
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Se van revisando las palabras de la sentencia
    for word in sentence:
        # Si la palabra es un numero, se agrega el token <NUM>
        if word.isnumeric():
            blogs_tokens_train.append('<NUM>')
        # Si la palabra no es vacia
        elif word != "":
            # Si la palabra termina en algo que no sea una letra o un numero
            # se quita el ultimo caracter de la palabra y se agregan las dos partes
            if not (word[-1].isalpha() or word[-1].isnumeric()):
                # Si la palabra restante es un numero, se agrega el token <NUM>
                if word[:-1].isnumeric():
                    blogs_tokens_train.append('<NUM>')
                # Si la palabra restante tiene frecuencia 1 o menos, se agrega el token <UNK>
                elif blogs_word_counts_train[word[:-1]] <= 1: # Se hace así por si la palabra restante tiene frecuencia 0 (no existe)
                    blogs_tokens_train.append('<UNK>')
                else:
                    blogs_tokens_train.append(word[:-1])
                blogs_tokens_train.append(word[-1])
            # Si su frecuencia es 1 o menos, se agrega el token <UNK>
            elif blogs_word_counts_train[word] <= 1:
                blogs_tokens_train.append('<UNK>')
            # Si su frecuencia es mayor a 1, se agrega la palabra
            else:
                blogs_tokens_train.append(word)
    # Se agrega el token de fin de oracion
    blogs_tokens_train.append('</s>')

print('Cantidad de tokens en BAC_GROUP_training: ', len(blogs_tokens_train))


Cantidad de palabras en BAC_GROUP_training:  1303444
Cantidad de tokens en BAC_GROUP_training:  134129367


Por facilidad se guarda un archivo con estos tokens

In [4]:
# Guardamos los tokens de train en un archivo .pickle
import pickle

with open('BAC_training_tokens.pickle', 'wb') as file:
    for token in blogs_tokens_train:
        pickle.dump(token, file)

In [5]:
print(blogs_tokens_train[:2000])

['<s>', "i've", 'been', 'selling', 'books', 'online', 'since', 'april', ',', 'and', 'am', 'painfully', 'aware', 'of', 'the', 'vast', 'ocean', 'of', 'book-knowledge', 'that', 'i', "haven't", 'yet', 'begun', 'to', 'cross', '</s>', '<s>', 'but', 'even', 'before', 'i', 'set', 'out', 'on', 'that', 'voyage', ',', "there's", 'a', 'smaller', 'body', 'of', 'retailing', 'commonsense', 'i', 'ought', 'to', 'master', '</s>', '<s>', 'alas', 'for', 'me', '</s>', '<s>', "who'd", 'have', 'guessed', 'that', "there's", 'a', 'person', 'out', 'there', 'who', 'uses', 'orange', 'highlighter', 'in', 'an', 'otherwise', 'lovely', 'urllink', 'heritage', 'press', 'edition', 'of', 'christopher', 'marlowe', '?', "what'd", 'they', 'think', 'it', 'was', ',', 'a', 'textbook', '?', 'lesson', '#1', ':', "don't", 'wait', 'until', "you're", 'ready', 'to', 'sell', 'the', 'merchandise', 'to', 'give', 'it', 'a', 'careful', 'inspection', ',', 'even', 'if', 'it', 'looks', 'great', 'on', 'the', 'outside', '</s>', '<s>', 'welcom

## **IV. Crear modelos de lenguaje**

In [1]:
from nltk.util import ngrams
from nltk.probability import FreqDist, LaplaceProbDist
import pickle

## Modelo de lenguaje de N-gramas - 20N

In [7]:
# Primero se crea el modelo de unigramas
news_unigrams_train = ngrams(news_tokens_train, 1)

# Se calculan las frecuencias de cada unigrama
news_unigrams_freq = FreqDist(news_unigrams_train)

# Se aplica suavizado laplaciano
news_unigrams_prob = LaplaceProbDist(news_unigrams_freq, bins=len(news_word_counts_train))

# Ejemplo de probabilidad de un unigrama
probabilidad = news_unigrams_prob.prob('I')
print(f'Probabilidad de "the": {probabilidad}')

print("Probabilidad más alta:",news_unigrams_prob.max())

# Guardar el modelo de unigramas
with open('20N_unigrams.pkl', 'wb') as file:
    pickle.dump(news_unigrams_prob, file)

Probabilidad de "the": 1.827705827091718e-07
Probabilidad más alta: ('<s>',)


In [8]:
# Crear el modelo de bigramas
news_bigrams_train = ngrams(news_tokens_train, 2)

# Se calculan las frecuencias de cada bigrama
news_bigrams_freq = FreqDist(news_bigrams_train)

# Se aplica suavizado laplaciano
news_bigrams_prob = LaplaceProbDist(news_bigrams_freq)

# Ejemplo de probabilidad de un bigrama
probabilidad = news_bigrams_prob.prob(('i', 'am'))
print(f'Probabilidad de "i am": {probabilidad}')

print("Probabilidad más alta:", news_bigrams_prob.max())

# Guardar el modelo de bigramas
with open('20N_bigrams.pkl', 'wb') as file:
    pickle.dump(news_bigrams_prob, file)

Probabilidad de "i am": 0.0005458112506051984
Probabilidad más alta: ('</s>', '<s>')


In [9]:
# Crear el modelo de trigramas
news_trigrams_train = ngrams(news_tokens_train, 3)

# Se calculan las frecuencias de cada trigramas
news_trigrams_freq = FreqDist(news_trigrams_train)

# Se aplica suavizado laplaciano
news_trigrams_prob = LaplaceProbDist(news_trigrams_freq)

# Ejemplo de probabilidad de un trigramas
probabilidad = news_trigrams_prob.prob(('i', 'am', 'a'))
print(f'Probabilidad de "i am a": {probabilidad}')

print("Probabilidad más alta:", news_trigrams_prob.max())

# Guardar el modelo de trigramas
with open('20N_trigrams.pkl', 'wb') as file:
    pickle.dump(news_trigrams_prob, file)

Probabilidad de "i am a": 1.7704921726809303e-05
Probabilidad más alta: ('<UNK>', '</s>', '<s>')


## Modelo de lenguaje de N-gramas - BAC

In [2]:
# Cargar el archivo de tokens de BAC
import pickle

with open('BAC_training_tokens.pickle', 'rb') as file:
    blogs_tokens_train = []
    while True:
        try:
            blogs_tokens_train.append(pickle.load(file))
        except EOFError:
            break

In [6]:
# Primero se crea el modelo de unigramas
blogs_unigrams_train = ngrams(blogs_tokens_train, 1)

# Se calculan las frecuencias de cada unigrama
blogs_unigrams_freq = FreqDist(blogs_unigrams_train)

# Se aplica suavizado laplaciano
blogs_unigrams_prob = LaplaceProbDist(blogs_unigrams_freq)

# Ejemplo de probabilidad de un unigrama
probabilidad = blogs_unigrams_prob.prob('i')
print(f'Probabilidad de "the": {probabilidad}')

print("probabilidad más alta:", blogs_unigrams_prob.max())

# Guardar el modelo de unigramas
with open('BAC_unigrams.pkl', 'wb') as file:
    pickle.dump(blogs_unigrams_prob, file)

Probabilidad de "the": 7.429830470660517e-09
probabilidad más alta: ('<s>',)


In [7]:
# Crear el modelo de bigramas
blogs_bigrams_train = ngrams(blogs_tokens_train, 2)

# Se calculan las frecuencias de cada bigrama
blogs_bigrams_freq = FreqDist(blogs_bigrams_train)

# Se aplica suavizado laplaciano
blogs_bigrams_prob = LaplaceProbDist(blogs_bigrams_freq)

# Ejemplo de probabilidad de un bigrama
probabilidad = blogs_bigrams_prob.prob(('i', 'am'))

print(f'Probabilidad de "i am": {probabilidad}')

print("probabilidad más alta:", blogs_bigrams_prob.max())

# Guardar el modelo de bigramas
with open('BAC_bigrams.pkl', 'wb') as file:
    pickle.dump(blogs_bigrams_prob, file)

Probabilidad de "i am": 0.001330170538731935
probabilidad más alta: ('</s>', '<s>')


In [3]:
# Crear el modelo de bigramas
blogs_bigrams_train = ngrams(blogs_tokens_train, 2)

# Se calculan las frecuencias de cada bigrama
blogs_bigrams_freq = FreqDist(blogs_bigrams_train)

# Se aplica suavizado laplaciano
blogs_bigrams_prob = LaplaceProbDist(blogs_bigrams_freq)

El modelo de trigramas no corrió :\( Se intentaron muchas formas, siendo la última la siguiente

In [4]:
# Crear el modelo de trigramas
blogs_trigrams_train = ngrams(blogs_tokens_train, 3)

# Se calculan las frecuencias de cada trigrama iterativamente para ahorrar memoria
freq_dist = {}
for trigram in blogs_trigrams_train:
    freq_dist[trigram] = freq_dist.get(trigram, 0) + 1

# Se aplica suavizado laplaciano iterativamente para ahorrar memoria y se guarda en un archivo .txt dinamicamente
with open('BAC_trigrams.txt', 'w') as file:
    trigram_prob = {}
    for trigram in blogs_trigrams_train:
        trigram_prob[trigram] = (freq_dist.get(trigram, 0) + 1) / (blogs_bigrams_freq[(trigram[0], trigram[1])] + len(freq_dist.keys()))
        file.write(str(trigram) + ' ' + str(trigram_prob[trigram]) + '\n')




: 